In [1]:
import time
import os
import cv2 as cv
import mediapipe as mp

In [2]:
# Module
# Hand Traching Module
class handDetector():
    def __init__(self , mode = False , maxHands = 2 , detectionCon = 0.5 , trackCon = 0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        self.mpHands= mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode , self.maxHands ,1, self.detectionCon , self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        
    def findHands(self,img,draw = True):
        imgRGB =  cv.cvtColor(img , cv.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw : self.mpDraw.draw_landmarks(img,handLms , self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self , img , handNo = 0 , draw = True):
        lmList = []
        if self.results.multi_hand_landmarks:
            
            myHand = self.results.multi_hand_landmarks[handNo]
            
            for id , lm in enumerate(myHand.landmark):
                    h, w ,c = img.shape
                    cx , cy = int(lm.x*w ) , int(lm.y * h)
                    lmList.append([id, cx, cy])
                    
                    if draw : cv.circle(img,(cx , cy) , 25 , (255,0,255) , cv.FILLED)
                    
        return lmList

In [3]:
# Constants
wCam , hCam = 640 , 480 # Cam  Width , Height

In [4]:
cap = cv.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)

pTime = 0

detector = handDetector(detectionCon=0.75)

tipIds = [4 , 8 , 12 , 16 , 20]
fingers = []

while True:
    success , img = cap.read()
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw = False)
    
    if len(lmList) != 0 :
        fingers = []
        
        # thums
        if lmList[tipIds[0]][1] < lmList[tipIds[0] - 1][1]:
            fingers.append(0)
        else:
            fingers.append(1)
            
        # Fingers
        for id in range(1,5):
            if lmList[tipIds[id]][2] < lmList[tipIds[id]-2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        
    
    cv.putText(img,str(fingers.count(1)) ,(20,150) , cv.FONT_HERSHEY_PLAIN , 10 , (255,0,0), 25)
    cTime = time.time()
    fbs = 1/(cTime - pTime)
    pTime = cTime
    cv.putText(img , f"FBS {int(fbs)}" ,(400,70) , cv.FONT_HERSHEY_COMPLEX , 2 , (255,0,0) , 2)
    
    cv.imshow("Image",img)
    cv.waitKey(1)

d:\Src\Anaconda\lib\site-packages\google\protobuf\symbol_database.py:78: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 

: 